In [60]:
import torch

#### We dont need to specify requires_grad = False, since by default it flags it as False

In [61]:
tensor1 = torch.Tensor([[1, 2, 3], 
                       [4, 5, 6]])
tensor1

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [62]:
tensor2 = torch.Tensor([[7, 8, 9], 
                        [10, 11, 12]])

tensor2

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.]])

#### The requires_grad property defines whether to track operations on this tensor
By default, it is set to False

In [63]:
tensor1.requires_grad

False

In [64]:
tensor2.requires_grad

False

#### The requires\_grad\_() function sets requires_grad to True

In [65]:
tensor1.requires_grad_()

tensor([[1., 2., 3.],
        [4., 5., 6.]], requires_grad=True)

In [66]:
tensor1.requires_grad

True

In [67]:
tensor2.requires_grad

False

#### The .grad property stores all the gradients for the tensor
However, there are no gradients yet

In [68]:
print(tensor1.grad)

None


#### The .grad_fn property contains the gradient function
This has not been set either

In [69]:
print(tensor1.grad_fn)

None


#### Create a new output tensor from our original tensor

In [70]:
output_tensor = tensor1 * tensor2

#### The requires_grad property has been derived from the original tensor

In [71]:
output_tensor.requires_grad

True

#### There are still no gradients

In [72]:
print(output_tensor.grad)

None


#### But there is a gradient function
This is from the multiplication operation performed on the original tensor 

In [73]:
print(output_tensor.grad_fn)

#### The original tensor still does not have a gradient function

In [74]:
print(tensor1.grad_fn)

None


In [75]:
print(tensor2.grad_fn)

None


#### Changing the operation for the output changes the gradient function
The gradient function only contains the last operation. Here, even though there is a multiplication as well as a mean, only the mean calculation is recorded as the gradient function

In [76]:
output_tensor = (tensor1 * tensor2).mean()
print(output_tensor.grad_fn)

#### In spite of setting a gradient function for the output, the gradients for the input tensor is still empty

In [77]:
print(tensor1.grad)

None


#### To calculate the gradients, we need to explicitly perform a backward propagation

In [78]:
output_tensor.backward()

#### The gradients are now available for the input tensor

Future calls to backward will accumulate gradients into this vector

In [80]:
print(tensor1.grad)

tensor([[1.1667, 1.3333, 1.5000],
        [1.6667, 1.8333, 2.0000]])


#### The gradient vector is the same shape as the original vector

In [82]:
tensor1.grad.shape, tensor1.shape

(torch.Size([2, 3]), torch.Size([2, 3]))

In [83]:
print(tensor2.grad)

None


In [84]:
print(output_tensor.grad)

None


#### The requires_grad property propagates to other tensors
Here the new_tensor is created from the original tensor and gets the original's value of requires_grad

In [85]:
new_tensor = tensor1 * 3
print(new_tensor.requires_grad)

True


In [86]:
new_tensor

tensor([[ 3.,  6.,  9.],
        [12., 15., 18.]], grad_fn=<MulBackward0>)

#### Turning off gradient calculations for tensors
You can also stops autograd from tracking history on newly created tensors with requires_grad=True by wrapping the code block in <br />
<b>with torch.no_grad():</b>

In [87]:
with torch.no_grad():
    
    new_tensor = tensor1 * 3
    
    print('new_tensor = ', new_tensor)
    
    print('requires_grad for tensor = ', tensor1.requires_grad)
    
    print('requires_grad for tensor = ', tensor2.requires_grad)
    
    print('requires_grad for new_tensor = ', new_tensor.requires_grad)

new_tensor =  tensor([[ 3.,  6.,  9.],
        [12., 15., 18.]])
requires_grad for tensor =  True
requires_grad for tensor =  False
requires_grad for new_tensor =  False


#### Can turn off gradient calculations performed within a function

In [88]:
def calculate(t):
    return t * 2

In [89]:
@torch.no_grad()
def calculate_with_no_grad(t):
    return t * 2

In [90]:
result_tensor = calculate(tensor1)

result_tensor

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.]], grad_fn=<MulBackward0>)

In [91]:
result_tensor.requires_grad

True

In [92]:
result_tensor_no_grad = calculate_with_no_grad(tensor1)

result_tensor_no_grad

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.]])

In [93]:
result_tensor_no_grad.requires_grad

False

#### Can explicitly enabled gradients within a no_grad() context

There is an equivalent @torch.enable_grad() as well

In [96]:
with torch.no_grad():
    
    new_tensor_no_grad = tensor1 * 3
    
    print('new_tensor_no_grad = ', new_tensor_no_grad)
    
    with torch.enable_grad():
        
        new_tensor_grad = tensor1 * 3
    
        print('new_tensor_grad = ', new_tensor_grad)

new_tensor_no_grad =  tensor([[ 3.,  6.,  9.],
        [12., 15., 18.]])
new_tensor_grad =  tensor([[ 3.,  6.,  9.],
        [12., 15., 18.]], grad_fn=<MulBackward0>)


### Result tensors get requires_grad properties from input tensors

In [98]:
tensor_one = torch.tensor([[1.0, 2.0], 
                           [3.0, 4.0]], requires_grad=True)  
tensor_one

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [99]:
tensor_two = torch.Tensor([[5, 6], 
                           [7, 8]])
tensor_two

tensor([[5., 6.],
        [7., 8.]])

#### enable the gradients for  two tensors

In [100]:
tensor_one.requires_grad

True

In [101]:
tensor_two.requires_grad_()

tensor([[5., 6.],
        [7., 8.]], requires_grad=True)

In [102]:
final_tensor = (tensor_one + tensor_two).mean()
final_tensor

tensor(9., grad_fn=<MeanBackward1>)

#### final tensor has gradients enabled as it derives from the tensors its made up of

In [103]:
final_tensor.requires_grad

True

In [104]:
print(tensor_one.grad)

None


In [105]:
print(tensor_two.grad)

None


In [106]:
final_tensor.backward()

In [107]:
print(tensor_one.grad)

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])


In [108]:
print(tensor_two.grad)

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])


In [109]:
print(final_tensor.grad)

None


#### Detach tensors from the computation graph

In [111]:
detached_tensor = tensor_one.detach()

detached_tensor

tensor([[1., 2.],
        [3., 4.]])

In [112]:
tensor_one

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [113]:
mean_tensor = (tensor_one + detached_tensor).mean()

mean_tensor.backward()

In [114]:
tensor_one.grad

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000]])

In [116]:
print(detached_tensor.grad)

None
